# Cars 4 You: Expending Car Evaluations with ML

Project Developed by:

*  Catarina Martins (n20221914)
*  Inês Van Zeller (n20222047)
*  Marta Ribeiro (n20221886)
*  Nicole Nogueira (n20221961)

## 1. Import the needed libraries

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
import numpy as np

: 

## 2. Data importation and integration

In [ ]:
train_data = pd.read_csv('project_data/train.csv')
test_data = pd.read_csv('project_data/test.csv')

In [ ]:
data= train_data.copy()

## 3. Data exploration and understanding

### Metadata
- *carID*: An attribute that contains an identifier for each car.
- *Brand*: The car’s main brand (e.g. Ford, Toyota).
- *model*: The car model.
- *year*: The year of Registration of the Car.
- *mileage*: The total reported distance travelled by the car (in
 miles).
- *tax*: The amount of road tax (in £) that, in 2020, was
 applicable to the car in question.
- *fuelType*: Type of Fuel used by the car (Diesel, Petrol, Hybrid,
 Electric).
- *mpg*: Average Miles per Gallon.
- *engineSize*: Size of Engine in liters (Cubic Decimeters).
- *paintQuality%*:  The mechanic’s assessment of the cars’ overall paint
 quality and hull integrity (filled by the mechanic
 during evaluation). 
- *previousOwners*: Number of previous registered owners of the vehicle.
- *hasDamage*:  Boolean marker filled by the seller at the time of
 registration stating whether the car is damaged or
 not.
- *price*: The car’s price when purchased by Cars 4 You (in £).

In [ ]:
#overview the dataset
data.info()

1: our data has 14 variables (columns), being 10 numerical variables (float and integers) 
and 4 non-numerical variables (strings). Moreover, one of the variables is the carID.

2: besides all the variables have observations, some variables like tax and mpg present a considerable number of missing values.

In [ ]:
#first 20 rows
data.head(20)

By acessing to the 20 first rows of the data, we can see:

1:There are some spelling mistakes in the data itself: 'for' instead of 'ford', anual' instead of 'manual' and 'etrol' instead of 'petrol', etc.

2:The variable 'previousOwners' that represents the number of previous registered owners of the vehicle, presents at least one negative value, which is impossible.

3:All the observations of the first 20 rows of the variable 'hasDamage' are 0.

In [ ]:
#last 20 rows
data.tail(20)

By acessing to the 20 first rows of the data, we can see that the spelling mistakes continue and the variable 'hasDamage' remain with all 0 observations. 

In [ ]:
#Descriptive statistics for numerical data
data.describe().T

For metric variables, we can see some metrics such as the mean, the minimum and maximum number, etc. to have a better inside of the data.

We can observe that the minimum number of 'mileage' is negative which is wrong because a car cannot travel negative miles. The same thing happens for the variables 'tax', 'mpg', 'engineSize', and 'previousOwners'.

In the 'previousOwners' and 'year' variables, the maximum numbers are floats which is a mistake because these variables should only assume integer numbers. The maximum of 'PaintQuality%' is more than 100%. The maximum of 'Mpg' is too high (it should be more or less 150)

As expected, all the observations of the variable 'hasDamage' are 0, which indicates that further ahead we are going to delete it because it does not add any information to our model. 


In [ ]:
#Descriptive statistics for categorical data
data.describe(include = ['O'])

For non-metric features, we can observe features like the unique values and the observations that appear the most in each variable.

We can understand that are some missing values in these variables but not a considerable quantity. 

Besides there are only 72 different brands, there are 735 different models. We can not forget that these values are biased due to the spelling mistakes, which means that the quantity of unique values are lower than showed. 

Moreover, the most frequent car in our dataset is a manual petrol ford focus. 

In [ ]:
#separate numerical and categorical features 

metric_features = ['year', 'mileage', 'tax', 'mpg',
                    'engineSize', 'paintQuality%', 'previousOwners', 'hasDamage']

non_metric_features= ['Brand','model','transmission','fuelType']

identifier = 'carID'

target = 'price'

In [ ]:
for col in non_metric_features:
    print(f"\nColumn: {col}")
    print(data[col].unique())

Brand:
'VW': 'V', 'vw', 'v'
'Toyota': 'Toyot', 'TOYOTA', 'oyota', 'toyota', 'OYOTA', 'TOYOT', 'toyot'
'Audi': 'udi', 'AUDI', 'audi', 'Aud', 'aud', 'UDI', 'AUD'
'Ford': 'FOR', 'ord', 'For', 'FORD', 'ford', 'for', 'or', 'ORD'
'BMW'. 'MW', 'bmw', 'BM', 'mw'
'Skoda': 'koda', 'skoda', 'SKODA', 'Skod', 'kod', 'SKOD', 'KODA', 'skod'  
'Opel': 'Ope', 'opel', 'pel', 'pe', ', OPEL', 'PEL', 'OPE', 'ope'
'Mercedes': 'mercedes', 'Mercede', 'MERCEDES', 'ercedes', 'mercede', 'ERCEDES' ,'ercede', 'MERCEDE'
'Hyundai': 'yundai', 'Hyunda', 'hyundai', 'HYUNDAI', 'yunda', 'HYUNDA', 'ud', 'hyunda'
'w','W' 
72

Model: 
'Golf': ' GOLF', ' Gol',' golf','golf',' Golf',' gol', ' GOL', 'Gol'
'Yaris': ' Yaris',' YARIS',' Yari',' yaris', ' yari','Yari', ' YARI', 'yaris' 
'Q2': ' q2',' Q2'
'2 Series':' 2 series', ' 2 serie', '2 Series',' 2 SERIES',' 2 Serie','2 Serie',' 2 Series'
'3 Series':' 3 Series',' 3 Serie',' 3 series',' 3 SERIES',' 3 serie', '3 Serie'
'A3':' A3',' a3'
'Octavia':' Octavi',' OCTAVIA',' Octavia',' octavia','Octavi', 'octavia'
'Passat': ' PASSAT',' passat',' Passa','Passat',' Passat'
'Insignia':' Insigni', ' INSIGNIA',' insignia',' Insignia',' INSIGNI', 'Insigni'
'Fabia':' Fabia', ' fabia', ' FABIA',' Fabi', 'FABIA'
'A Class':' A Clas',' A Class' ,' a class',' A CLASS','a class', 'A CLASS', ' a clas'
'Ka+': ' Ka+',' KA+',' ka+', 'ka+'
'GLC Class': ' GLC Class',' GLC CLASS',' GLC Clas',' glc class', ' glc clas'
'I30':' i30',' I30' 
'C Class': ' C Class':' C Clas',' C CLASS', ' c class','C Clas', ' C CLAS', 'c class',' c clas' 
'Polo':' POLO', ' Polo'
'E Class':' E Class',' E Clas',' E CLASS',' e class','E CLASS', 'e class'
'Q5': ' Q5',' q5', 'q5'
'Up':'U',' up',' UP',' Up',' U', 'UP'
'Fiesta':' FIESTA',' fiesta',' Fiest',' Fiesta', 'fiesta','Fiest', 'FIESTA'
'C-HR':' C-H',' c-hr',' C-HR'
'Mokka X':' mokka x', ' MOKKA X',' Mokka X'
'Corsa':' Corsa',' corsa',' Cors',' CORSA',' cors', ' CORS','corsa'
'Astra':' ASTRA', ' Astr', ' Astra', ' astra', 'ASTRA', 'astra'
'TT':' tt',' TT', ' T'
'5 Series': ' 5 Series', ' 5 Serie', ' 5 SERIES',' 5 series', '5 SERIES'
'Aygo': ' aygo',' ayg',' AYGO',' Ayg',' Aygo','aygo', ' AYG'
'4 Series':' 4 SERIES', ' 4 Serie', ' 4 serie', '4 series','4 Series',' 4 Series',' 4 series'
'SLK':' slk',' SLK'
'Viva':' viva',' Viva',' VIVA',' Viv' 
'Focus':' Focus',' Focu',' FOCUS',' focus',' FOCU', 'focus','Focu', ' focu', 'FOCUS'
'EcoSport':' EcoSpor', ' ECOSPORT',' ecosport',' EcoSport'
'X-CLASS':' x-clas',' X-CLAS',' x-class', ' X-CLASS'
'CL Class': ' cl class',' CL Clas',' CL CLASS', ' CL Class'
'IX20':' ix20', ' IX20'
'Rapid': ' Rapi',' rapid', ' Rapid'
'Auris':' Auris', ' AURIS',' auris',' Auri'
'I20':' i20',' I20'
'X3':' x3',' X3' 
'A8':' A8', 'a8'
'GLS Class':' GLS Clas',' GLS CLASS', ' gls class', ' GLS Class'
'B-MAX': ' B-MA',' B-MAX','B-MA',' b-max'
'A4': ' A4', ' a4'
'Kona':' KONA',' Kon',' Kona', ' KON', ' kona'
'I10':' i10',' I10'
'A1':' A1',' a1',
'Mokka': ' Mokka ', ' Mokk',' Mokka',' mokka ',' mokka',' MOKKA', 'Mokka ', 'Mokk'
'S-MAX': ' S-MA',' s-max',' S-MAX', ' s-ma'
'X2': ' x2',' X2'
'Crossland X': ' crossland x', ' CROSSLAND X',' Crossland X'
'Tiguan':' Tiguan',' tiguan',' Tigua',' TIGUAN',' TIGUA', 'Tigua', 'TIGUAN', 'tiguan'
'A5': ' A5', ' a5'
'GLE Class':' GLE Clas',' GLE Class',' gle class', ' GLE CLASS'
'Zafira':' Zafira', ' Zafir',' ZAFIRA',' zafira','Zafir'
'Ioniq':' Ioni',' Ioniq', ' IONIQ', 'IONIQ', ' ioniq'
'A6':' A6',' a6'
'Yeti Outdoor': ' yeti outdoor',' Yeti Outdoor' 
'X1':' x1','x1',' X1'
'Polo': 'POLO', ' polo', ' Pol','polo','Pol'
'Scala':' SCALA',' Scala',' scala', ' Scal', ' scal'
'S Class':' S Class',' S Clas',' s class',' S CLASS'
'1 Series':' 1 Series',' 1 SERIES', ' 1 Serie',' 1 series', '1 SERIES',' 1 SERIE', '1 series', ' 1 serie'
'Kamiq': ' KAMIQ',' KAMI', ' kamiq', ' Kamiq'
'Kuga': ' Kug', ' KUGA', ' kuga','Kuga',' Kuga', 'kuga'
'Tourneo Connect': ' tourneo connect', ' Tourneo Connect'
'Q7': ' q7',' Q7'
'GLA Class':' GLA Class',' GLA CLASS',' GLA Clas',' gla class'
'Arteon': ' arteon', ' Arteon'
'SL CLASS':' SL CLAS',' SL CLASS', ' sl class', ' SL'
'Tucson': ' Tucson',' TUCSON',' Tucso',' tucson',' TUCSO', 'Tucso'
'Santa Fe':' Santa F',' santa fe',' SANTA FE',' Santa Fe'
'Grandland X':' Grandland X',' grandland x', ' GRANDLAND X'
' I800':
'RAV4': ' rav4','RAV4','RAV',' RAV','rav4', ' rav', ' RAV4'
'Touran': ' Touran', 'Toura', ' TOURAN',' touran',' Toura', ' TOURA'
'Citigo': ' Citig',' citigo',' Citigo',' CITIGO'
'Roomster':' Roomste', ' Roomster'
'Prius':' PRIUS',' Prius', 'Prius',' prius'
'Corolla': ' corolla',' COROLLA', ' Coroll',' Corolla', 'corolla'
'B Class': ' b class', ' B Clas',' B Class',  ' B CLASS', 'b class'
'Sharan':' sharan',' Shara', ' Sharan', ' SHARAN'
' Q':
'Kodiaq':' Kodia', ' kodiaq','kodiaq',' KODIAQ', ' Kodiaq'
'V Class': ' V Clas',' V CLASS', ' V Class'
'Caddy Maxi Life':' Caddy Maxi Lif', ' Caddy Maxi Life'
'Superb':' Superb',' Super',' SUPERB',' superb',' super'
' Getz',
'T-Roc':' T-Roc',' T-RO',' t-roc',' T-Ro',' T-ROC'
'Combo Life':' COMBO LIFE',' combo life', ' Combo Lif', 'COMBO LIFE', ' Combo Life'
'Beetle':' Beetl',' Beetle', ' beetle'
'Galaxy': ' GALAXY',' Galax',' galaxy',' Galaxy'  
'M3':' M3', ' m3' 
'GTC': 'gtc',' GTC', ' gtc'
'X4':' X4'
'KA': ' Ka',' ka',' K', ' KA'
'IX35': ' ix35',' IX35' 
'Grand Tourneo Connect': ' Grand Tourneo Connec', ' Grand Tourneo Connect'
'M4':' m4', ' M4'
'Tourneo Custom':' tourneo custom', ' Tourneo Custo', ' Tourneo Custom'
'Z4':' Z4', ' z4'
'X5': ' X5',' x5' 
'Meriva': ' Meriva',' MERIVA',' Meriv', ' meriva'
'yeti outdoor': ' YETI OUTDOOR',' Yeti Outdoo', ' yeti outdoor'
'RS6':' RS6'
'Verso': ' VERSO',' verso',' Verso', ' Vers'
'Touareg': ' Touareg',' TOUAREG',' touareg', ' Touare'
'Mondeo':' MONDEO',' Mondeo',' mondeo',' MONDE',' Monde'
'Shuttle':' shuttle', ' Shuttle'
'CLS Class':' CLS Class', ' cls class',' CLS Clas', ' CLS CLASS'
'C-MAX': ' C-MAX',' c-max',' C-MA'
'Puma': ' puma',' PUMA', ' Puma'
'CLA Class':' CLA Class', ' CLA CLASS', ' cla class'
'I40':' I40',' i40'
'Q3': ' q3',' Q3'
'Tiguan Allspace':' TIGUAN ALLSPACE',' tiguan allspace', ' Tiguan Allspac', ' Tiguan Allspace'
'6 Series': ' 6 SERIES', ' 6 series',' 6 Series', ' 6 Serie'
'Caravelle':' caravelle',' Caravell', ' Caravelle'
'Karoq': ' Karoq',' karoq',' KAROQ', ' Karo'
'I3':' i3', 'i3', ' I3'
'Grand C-MAX':' GRAND C-MAX',' grand c-max',' Grand C-MA', ' Grand C-MAX'
'T-Cross': ' T-Cros',' T-CROSS', ' T-Cross',' t-cros', ' t-cross'
'A7':' a7',' A7' 
'Golf SV':' Golf SV',' golf sv', ' GOLF SV'
'A':' a', ' A' 
' Agila':
' GT86': ' gt86'
'Yeti': ' yeti', ' Yet', ' Yeti'
' California':
'X':' x', ' X'
'Land Cruiser': ' Land Cruise, ' Land Cruiser'
'EDGE': ' Edge',' edge', ' Edg', ' EDGE'
'X6':' X6'
'Fusion':' Fusion' 
'GL Class': ' GL CLASS', ' gl class',' GL Class' 
'Scirocco':' scirocco',' SCIROCCO',' Scirocc',' Scirocco'
'Z3':' Z3'
' PROACE VERSO':
'Hilux':' hilux',' Hilux', ' Hilu'
'Amarok':' amarok',' Amarok', ' Amaro'
'CC':' cc', ' CC'
'7 Series':' 7 Serie', ' 7 SERIES', ' 7 series', ' 7 Series'
'Avensis':' AVENSIS', ' avensis', ' Avensis'
' Eos':
'M Class':' m class', ' M CLASS', ' M Class', ' M Clas'
'Grandland ':' grandland ', ' Grandland '
'Zafira Tourer':' Zafira Toure', ' ZAFIRA TOURER', ' Zafira Tourer'
' RS5':
'R8':' R8', ' r8'
'Mustang':' mustang',' Mustang'
' I1':
' Antara':
'Q8':' Q8' 
' Camry':
'CLK':' CLK' 
'RS3':' RS3' 
'Jetta': ' JETTA', ' Jetta'
' Kadjar':
' SQ5': 
' RS4':
'Supra':' Supra'
' i8':
'X7':' X7'
' sq7':' SQ7'
' G Class':
' S3':' s3' 
' I80':
' Crossland ':
'Arteon':' Arteo', 'Arteo' , ' ARTEON'
' Tigra':
' Escort':
'GLB Class': ' glb class', ' GLB Class'
' Vivaro':
' Verso-S':
'Adam': ' Ada',' adam', ' ADAM', ' Adam'
'M5':' M5' 
' Veloste':
' S4':
' IQ': 
' A2':
' Caddy Maxi':
'200':
' Streetka':
' v class': 
' Caddy':   
' Cascada':
' Accent':
' S8':    
' RS': 
'Golf S':' golf s', ' Golf S'
' Z':
' Ranger':
' I2':
' M':
'220':
'VECTRA': 'Vectra',' Vectra', ' VECTRA'
' 8 Serie': ' 8 SERIES'
'230':  
' Ampera':
' Fox':  
'Urban Cruiser': ' Urban Cruise', ' Urban Cruiser'
' M2':   
' S5':    
' IX2':
' CLC Class' 
' M6':
' Terracan':  
' i':   
   

transmission:
'Semi-Auto':'Semi-Aut','semi-auto','emi-Auto', 'SEMI-AUTO','SEMI-AUT','EMI-AUTO','emi-Aut','emi-auto','semi-aut'
'Manual':'anual', 'manual','Manua','MANUAL',' Manual ','ANUAL','manua','anua','MANUA',' manual ',' MANUAL ',' Manual','Manual ','manual '
'Automatic':'AUTOMATIC', 'automatic','Automati','utomatic','UTOMATIC','automati','AUTOMATI','utomati' 
'unknown':'unknow','UNKNOWN','nknown','nknow'
'Other'     
      
   
   
fuelType:  
'Petrol':'etrol','petrol','PETROL','Petro','petro','ETROL','PETRO','etro','ETRO'
'Diesel':'diesel','iesel','Diese','DIESEL','DIESE','IESEL','iese','diese','IESE'
'Hybrid':'HYBRID','ybri','Hybri','ybrid','hybrid','YBRID','HYBRI'
'Electric':
nan
'Other':'ther','Othe','OTHER','other'
   

### Checking memory usage and data types

In [ ]:
#checking memory usage
data.info(memory_usage='deep')

### Checking duplicates

In [ ]:
#checking duplicates
data.duplicated().sum()

There are no duplicates in the current dataset.  

### Checking missing values

In [ ]:
#Number of missing values
data.isna().sum()

The variables that have more missing values are 'tax' and 'mpg'. The rest of them have basically the same number of missing values, excluding 'carId', 'price'.

In [ ]:
#Number of missing values in each column as a percentage
data.isna().sum()/len(data) * 100

### Checking outliers

In [ ]:
## Define a function that plots multiple box plots

def plot_multiple_boxplots(data, feats, title="Numeric Variables' Box Plots"):

    # Prepare figure. Create individual axes where each histogram will be placed
    fig, axes = plt.subplots(4, ceil(len(feats) / 4), figsize=(40, 30))

    # Plot data
    # Iterate across axes objects and associate each histogram (hint: use the ax.hist() instead of plt.hist()):
    for ax, feat in zip(axes.flatten(), feats): # Notice the zip() function and flatten() method
      sns.boxplot(x=data[feat], ax=ax)
      ax.set_title(feat)

    # Layout
    # Add a centered title to the figure:
    plt.suptitle(title)
    #fig, ax = plt.subplots()
    #ax.boxplot(feat,
             #  patch_artist= True,
             #  boxprops= dict(facecolor = "lightblue"))
    plt.show()

    return

In [ ]:
plot_multiple_boxplots(data, metric_features)

We have a considerable number of outliers in all variables, except of 'paintQuality%', 'previousOwners' and 'hasDamage'. In the varibles 'mileage', 'mpg', 'engineSize', 'previousOwners', the negative outliers are incoherences so they are probably going to be removed. 

In [ ]:
df= pd.DataFrame(data[metric_features])

In [ ]:
num_cols = df.select_dtypes(include=['number']).columns
n = len(num_cols)

# Ajusta layout
fig, axes = plt.subplots(nrows=(n // 3) + 1, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    axes[i].hist(df[col].dropna(), bins=20, color='teal', edgecolor='black')
    axes[i].set_title(col)
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Frequência')

# Remove eixos vazios (se sobrarem)
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In terms of distribution, most variables are skewed either to the left ('mileage') or to the right ('year'). The closest to a normal distribution is identified in 'PaintQuality%'.

In [ ]:
sns.set_style('dark')
sns.set_palette("Paired")

# Create 4 subplots stacked vertically
fig, ax = plt.subplots(nrows=4, ncols=1, dpi=300, figsize=(20, 40))

# Plot each variable in its own row
sns.countplot(data=test_data, x='Brand', ax=ax[0])
sns.countplot(data=test_data, x='model', ax=ax[1])
sns.countplot(data=test_data, x='transmission', ax=ax[2])
sns.countplot(data=test_data, x='fuelType', ax=ax[3])

# Improve spacing between plots
plt.tight_layout()
plt.show()

The high amount of spelling mistakes leads to a lot of categories in each variable. One solution to this is merging the categories which would improve the readability of the plots. 

In [ ]:
cor_spearman = data[metric_features].corr(method ='spearman')
cor_spearman

In [ ]:
def cor_heatmap(cor):
    plt.figure(figsize=(12,10))
    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Reds, fmt='.1')
    plt.show()

In [ ]:
cor_heatmap(cor_spearman)

According to this correlation matrix, the variable 'mileage' is the only one that has a high negative correlation with year (<0.80), noting that we have to pre process the data in order to get reliable insides. 

In [ ]:
# Pairwise Relationship of Numerical Variables
sns.set()

# Setting pairplot
sns.pairplot(df[metric_features], diag_kind="hist")

# Layout
plt.subplots_adjust(top=0.95)
plt.suptitle("Pairwise Relationship of Numerical Variables", fontsize=20)

# if the eda directory is not present then create it first
if not os.path.exists(os.path.join('..', 'figures', 'eda')):
    os.makedirs(os.path.join('..', 'figures', 'eda'))
    
plt.savefig(os.path.join('..', 'figures', 'eda', 'pairwise_numeric_scatterplots.png'), dpi=200)
plt.show()

## 4. Data preparation

### Set index

In [ ]:
data.set_index('carID', inplace = True)

In [ ]:
data

### Changing datatypes

In [ ]:
data.info()

In [ ]:
data['year'] = data['year'].round().astype('Int64')
data['previousOwners'] = data['previousOwners'].round().astype('Int64')

In [ ]:
data = data.astype({'Brand' : 'object',
                    'model' : 'object',
                    'year' : 'Int64',
                    'transmission' : 'object',
                    'mileage' : 'float64',
                    'fuelType' : 'object',
                    'tax' : 'float64',
                    'mpg' : 'float64',
                    'engineSize' : 'float64',
                    'paintQuality%': 'float64',
                    'previousOwners' : 'Int64',
                    'hasDamage' : 'bool'})

In [ ]:
data.info()

The datatypes of the variables 'year' and 'previousOwners' and 'hasDamage' were changed into 'Int64', 'Int64' and 'bool' respectively. Also, the memory usage decreased to 7.8+ MB. 

In [ ]:
data.head(20)

It is important to note that, even though, 'previousOwners' no longer has decimal values, it still has negative numbers that must be handled. 

### Slipt the data into train and validation dataset

#### The Hold Out Method

In [ ]:
X = data.drop('price', axis = 1)
y = data['price']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.3, 
                                                  random_state = 0, 
                                                  shuffle = True)

In [ ]:
X_train

In [ ]:
X_val

In [ ]:
y_train

In [ ]:
y_val

### Handling Incoherencies

#### Correction of spelling mistakes for categorical variables:

In [ ]:
#corrections of 'brand' for X_train

correct_brand = {
    'VW': ['V', 'vw', 'v', 'W', 'w'],
    'Toyota': ['Toyot', 'TOYOTA', 'oyota', 'toyota', 'OYOTA', 'TOYOT', 'toyot'],
    'Audi': ['udi', 'AUDI', 'audi', 'Aud', 'aud', 'UDI', 'AUD'],
    'Ford': ['FOR', 'ord', 'For', 'FORD', 'ford', 'for', 'or', 'ORD'],
    'BMW': ['MW', 'bmw', 'BM', 'mw'],
    'Skoda': ['koda', 'skoda', 'SKODA', 'Skod', 'kod', 'SKOD', 'KODA', 'skod'],
    'Opel': ['Ope', 'opel', 'pel', 'pe', 'OPEL', 'PEL', 'OPE', 'ope'],
    'Mercedes': ['mercedes', 'Mercede', 'MERCEDES', 'ercedes', 'mercede', 'ERCEDES', 'ercede', 'MERCEDE'],
    'Hyundai': ['yundai', 'Hyunda', 'hyundai', 'HYUNDAI', 'yunda', 'HYUNDA', 'ud', 'hyunda']
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_brand.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_train["Brand"] = X_train["Brand"].replace(replacement_dict)

# Optional: verify the cleaning
print(X_train["Brand"].unique())

# Optional: save the cleaned dataset
# df.to_csv("train_cleaned.csv", index=False)

In [ ]:
#corrections of 'brand' for X_val

correct_brand = {
    'VW': ['V', 'vw', 'v', 'W', 'w'],
    'Toyota': ['Toyot', 'TOYOTA', 'oyota', 'toyota', 'OYOTA', 'TOYOT', 'toyot'],
    'Audi': ['udi', 'AUDI', 'audi', 'Aud', 'aud', 'UDI', 'AUD'],
    'Ford': ['FOR', 'ord', 'For', 'FORD', 'ford', 'for', 'or', 'ORD'],
    'BMW': ['MW', 'bmw', 'BM', 'mw'],
    'Skoda': ['koda', 'skoda', 'SKODA', 'Skod', 'kod', 'SKOD', 'KODA', 'skod'],
    'Opel': ['Ope', 'opel', 'pel', 'pe', 'OPEL', 'PEL', 'OPE', 'ope'],
    'Mercedes': ['mercedes', 'Mercede', 'MERCEDES', 'ercedes', 'mercede', 'ERCEDES', 'ercede', 'MERCEDE'],
    'Hyundai': ['yundai', 'Hyunda', 'hyundai', 'HYUNDAI', 'yunda', 'HYUNDA', 'ud', 'hyunda']
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_brand.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_val["Brand"] = X_val["Brand"].replace(replacement_dict)

# Optional: verify the cleaning
print(X_val["Brand"].unique())

# Optional: save the cleaned dataset
# df.to_csv("train_cleaned.csv", index=False)

In [ ]:
#correction of 'model' for X_train
correct_model = {
    'Golf': [' GOLF', ' Gol', ' golf', 'golf', ' Golf', ' gol', ' GOL', 'Gol'],
    'Yaris': [' Yaris', ' YARIS', ' Yari', ' yaris', ' yari', 'Yari', ' YARI', 'yaris'],
    'Q2': [' q2', ' Q2'],
    '2 Series': [' 2 series', ' 2 serie', '2 Series', ' 2 SERIES', ' 2 Serie', '2 Serie', ' 2 Series'],
    '3 Series': [' 3 Series', ' 3 Serie', ' 3 series', ' 3 SERIES', ' 3 serie', '3 Serie'],
    'A3': [' A3', ' a3'],
    'Octavia': [' Octavi', ' OCTAVIA', ' Octavia', ' octavia', 'Octavi', 'octavia'],
    'Passat': [' PASSAT', ' passat', ' Passa', 'Passat', ' Passat'],
    'Insignia': [' Insigni', ' INSIGNIA', ' insignia', ' Insignia', ' INSIGNI', 'Insigni'],
    'Fabia': [' Fabia', ' fabia', ' FABIA', ' Fabi', 'FABIA'],
    'A Class': [' A Clas', ' A Class', ' a class', ' A CLASS', 'a class', 'A CLASS', ' a clas'],
    'Ka+': [' Ka+', ' KA+', ' ka+', 'ka+'],
    'GLC Class': [' GLC Class', ' GLC CLASS', ' GLC Clas', ' glc class', ' glc clas'],
    'I30': [' i30', ' I30'],
    'C Class': [' C Clas', ' C CLASS', ' c class', 'C Clas', ' C CLAS', 'c class', ' c clas', ' C Class'],
    'Polo': [' POLO', ' Polo'],
    'E Class': [' E Class', ' E Clas', ' E CLASS', ' e class', 'E CLASS', 'e class'],
    'Q5': [' Q5', ' q5', 'q5'],
    'Up': ['U', ' up', ' UP', ' Up', ' U', 'UP'],
    'Fiesta': [' FIESTA', ' fiesta', ' Fiest', ' Fiesta', 'fiesta', 'Fiest', ' FIESTA'],
    'C-HR': [' C-H', ' c-hr', ' C-HR'],
    'Mokka X': [' mokka x', ' MOKKA X', ' Mokka X'],
    'Corsa': [' Corsa', ' corsa', ' Cors', ' CORSA', ' cors', ' CORS', 'corsa'],
    'Astra': [' ASTRA', ' Astr', ' Astra', ' astra', 'ASTRA', 'astra'],
    'TT': [' tt', ' TT', ' T'],
    '5 Series': [' 5 Series', ' 5 Serie', ' 5 SERIES', ' 5 series', '5 SERIES'],
    'Aygo': [' aygo', ' ayg', ' AYGO', ' Ayg', ' Aygo', 'aygo', ' AYG'],
    '4 Series': [' 4 SERIES', ' 4 Serie', ' 4 serie', '4 series', '4 Series', ' 4 Series', ' 4 series'],
    'SLK': [' slk', ' SLK'],
    'Viva': [' viva', ' Viva', ' VIVA', ' Viv'],
    'Focus': [' Focus', ' Focu', ' FOCUS', ' focus', ' FOCU', 'focus', 'Focu', ' focu', 'FOCUS'],
    'EcoSport': [' EcoSpor', ' ECOSPORT', ' ecosport', ' EcoSport'],
    'X-CLASS': [' x-clas', ' X-CLAS', ' x-class', ' X-CLASS'],
    'CL Class': [' cl class', ' CL Clas', ' CL CLASS', ' CL Class'],
    'IX20': [' ix20', ' IX20'],
    'Rapid': [' Rapi', ' rapid', ' Rapid'],
    'Auris': [' Auris', ' AURIS', ' auris', ' Auri'],
    'I20': [' i20', ' I20'],
    'X3': [' x3', ' X3'],
    'A8': [' A8', 'a8'],
    'GLS Class': [' GLS Clas', ' GLS CLASS', ' gls class', ' GLS Class'],
    'B-MAX': [' B-MA', ' B-MAX', 'B-MA', ' b-max'],
    'A4': [' A4', ' a4'],
    'Kona': [' KONA', ' Kon', ' Kona', ' KON', ' kona'],
    'I10': [' i10', ' I10'],
    'A1': [' A1', ' a1'],
    'Mokka': [' Mokka ', ' Mokk', ' Mokka', ' mokka ', ' mokka', ' MOKKA', 'Mokka ', 'Mokk'],
    'S-MAX': [' S-MA', ' s-max', ' S-MAX', ' s-ma'],
    'X2': [' x2', ' X2'],
    'Crossland X': [' crossland x', ' CROSSLAND X', ' Crossland X'],
    'Tiguan': [' Tiguan', ' tiguan', ' Tigua', ' TIGUAN', ' TIGUA', 'Tigua', 'TIGUAN', 'tiguan'],
    'A5': [' A5', ' a5'],
    'GLE Class': [' GLE Clas', ' GLE Class', ' gle class', ' GLE CLASS'],
    'Zafira': [' Zafira', ' Zafir', ' ZAFIRA', ' zafira', 'Zafir'],
    'Ioniq': [' Ioni', ' Ioniq', ' IONIQ', 'IONIQ', ' ioniq'],
    'A6': [' A6', ' a6'],
    'Yeti Outdoor': [' yeti outdoor', ' Yeti Outdoor'],
    'X1': [' x1', 'x1', ' X1'],
    'Scala': [' SCALA', ' Scala', ' scala', ' Scal', ' scal'],
    'S Class': [' S Class', ' S Clas', ' s class', ' S CLASS'],
    '1 Series': [' 1 Series', ' 1 SERIES', ' 1 Serie', ' 1 series', '1 SERIES', ' 1 SERIE', '1 series', ' 1 serie'],
    'Kamiq': [' KAMIQ', ' KAMI', ' kamiq', ' Kamiq'],
    'Kuga': [' Kug', ' KUGA', ' kuga', 'Kuga', ' Kuga', 'kuga'],
    'Tourneo Connect': [' tourneo connect', ' Tourneo Connect'],
    'Q7': [' q7', ' Q7'],
    'GLA Class': [' GLA Class', ' GLA CLASS', ' GLA Clas', ' gla class'],
    'Arteon': [' arteon', ' Arteon'],
    'SL CLASS': [' SL CLAS', ' SL CLASS', ' sl class', ' SL'],
    'Tucson': [' Tucson', ' TUCSON', ' Tucso', ' tucson', ' TUCSO', 'Tucso'],
    'Santa Fe': [' Santa F', ' santa fe', ' SANTA FE', ' Santa Fe'],
    'Grandland X': [' Grandland X', ' grandland x', ' GRANDLAND X'],
    'RAV4': [' rav4', 'RAV4', 'RAV', ' RAV', 'rav4', ' rav', ' RAV4'],
    'Touran': [' Touran', 'Toura', ' TOURAN', ' touran', ' Toura', ' TOURA'],
    'Citigo': [' Citig', ' citigo', ' Citigo', ' CITIGO'],
    'Roomster': [' Roomste', ' Roomster'],
    'Prius': [' PRIUS', ' Prius', 'Prius', ' prius'],
    'Corolla': [' corolla', ' COROLLA', ' Coroll', ' Corolla', 'corolla'],
    'B Class': [' b class', ' B Clas', ' B Class', ' B CLASS', 'b class'],
    'Sharan': [' sharan', ' Shara', ' Sharan', ' SHARAN'],
    'Kodiaq': [' Kodia', ' kodiaq', 'kodiaq', ' KODIAQ', ' Kodiaq'],
    'V Class': [' V Clas', ' V CLASS', ' V Class'],
    'Caddy Maxi Life': [' Caddy Maxi Lif', ' Caddy Maxi Life'],
    'Superb': [' Superb', ' Super', ' SUPERB', ' superb', ' super'],
    'T-Roc': [' T-Roc', ' T-RO', ' t-roc', ' T-Ro', ' T-ROC'],
    'Combo Life': [' COMBO LIFE', ' combo life', ' Combo Lif', 'COMBO LIFE', ' Combo Life'],
    'Beetle': [' Beetl', ' Beetle', ' beetle'],
    'Galaxy': [' GALAXY', ' Galax', ' galaxy', ' Galaxy'],
    'M3': [' M3', ' m3'],
    'GTC': [' gtc', ' GTC', ' gtc'],
    'X4': [' X4'],
    'KA': [' Ka', ' ka', ' K', ' KA'],
    'IX35': [' ix35', ' IX35'],
    'Grand Tourneo Connect': [' Grand Tourneo Connec', ' Grand Tourneo Connect'],
    'M4': [' m4', ' M4'],
    'Tourneo Custom': [' tourneo custom', ' Tourneo Custo', ' Tourneo Custom'],
    'Z4': [' Z4', ' z4'],
    'X5': [' X5', ' x5'],
    'Meriva': [' Meriva', ' MERIVA', ' Meriv', ' meriva'],
    'yeti outdoor': [' YETI OUTDOOR', ' Yeti Outdoo', ' yeti outdoor', 'yeti outdoor'],
    'RS6': [' RS6'],
    'Verso': [' VERSO', ' verso', ' Verso', ' Vers'],
    'Touareg': [' Touareg', ' TOUAREG', ' touareg', ' Touare'],
    'Mondeo': [' MONDEO', ' Mondeo', ' mondeo', ' MONDE', ' Monde'],
    'Shuttle': [' shuttle', ' Shuttle'],
    'CLS Class': [' CLS Class', ' cls class', ' CLS Clas', ' CLS CLASS'],
    'C-MAX': [' C-MAX', ' c-max', ' C-MA'],
    'Puma': [' puma', ' PUMA', ' Puma'],
    'CLA Class': [' CLA Class', ' CLA CLASS', ' cla class'],
    'I40': [' I40', ' i40'],
    'Q3': [' q3', ' Q3'],
    'Tiguan Allspace': [' TIGUAN ALLSPACE', ' tiguan allspace', ' Tiguan Allspac', ' Tiguan Allspace'],
    '6 Series': [' 6 SERIES', ' 6 series', ' 6 Series', ' 6 Serie'],
    'Caravelle': [' caravelle', ' Caravell', ' Caravelle'],
    'Karoq': [' Karoq', ' karoq', ' KAROQ', ' Karo'],
    'I3': [' i3', 'i3', ' I3'],
    'Grand C-MAX': [' GRAND C-MAX', ' grand c-max', ' Grand C-MA', ' Grand C-MAX'],
    'T-Cross': [' T-Cros', ' T-CROSS', ' T-Cross', ' t-cros', ' t-cross'],
    'A7': [' a7', ' A7'],
    'Golf SV': [' Golf SV', ' golf sv', ' GOLF SV'],
    'A': [' a', ' A'],
    'GT86': [' gt86', ' GT86'],
    'Yeti': [' yeti', ' Yet', ' Yeti'],
    'X': [' x', ' X'],
    'Land Cruiser': [' Land Cruise', ' Land Cruiser'],
    'EDGE': [' Edge', ' edge', ' Edg', ' EDGE'],
    'X6': [' X6'],
    'Fusion': [' Fusion'],
    'GL Class': [' GL CLASS', ' gl class', ' GL Class'],
    'Scirocco': [' scirocco', ' SCIROCCO', ' Scirocc', ' Scirocco'],
    'Z3': [' Z3'],
    'Hilux': [' hilux', ' Hilux', ' Hilu'],
    'Amarok': [' amarok', ' Amarok', ' Amaro'],
    'CC': [' cc', ' CC'],
    '7 Series': [' 7 Serie', ' 7 SERIES', ' 7 series', ' 7 Series'],
    'Avensis': [' AVENSIS', ' avensis', ' Avensis'],
    'M Class': [' m class', ' M CLASS', ' M Class', ' M Clas'],
    'Grandland': [' grandland ', ' Grandland '],
    'Zafira Tourer': [' Zafira Toure', ' ZAFIRA TOURER', ' Zafira Tourer'],
    'R8': [' R8', ' r8'],
    'Mustang': [' mustang', ' Mustang'],
    'Q8': [' Q8'],
    'CLK': [' CLK'],
    'RS3': [' RS3'],
    'Jetta': [' JETTA', ' Jetta'],
    'Supra': [' Supra'],
    'X7': [' X7'],
    'SQ7': [' SQ7'],
    'S3': [' s3'],
    'Arteon': [' Arteo', 'Arteo', ' ARTEON'],
    'GLB Class': [' glb class', ' GLB Class'],
    'Adam': [' Ada', ' adam', ' ADAM', ' Adam'],
    'M5': [' M5'],
    'Golf S': [' golf s', ' Golf S'],
    'VECTRA': ['Vectra', ' Vectra', ' VECTRA'],
    '8 Series': [' 8 SERIES', ' 8 Serie', ' 8 Series'],
    'Urban Cruiser': [' Urban Cruise', ' Urban Cruiser'],
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_model.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_train["model"] = X_train["model"].replace(replacement_dict)

# Optional: verify the cleaning
print(X_val["model"].unique())

#for model in sorted(X_train["model"].dropna().astype(str).unique()):
   # print(model)

In [ ]:
#correction of 'model' for X_val

correct_model = {
    'Golf': [' GOLF', ' Gol', ' golf', 'golf', ' Golf', ' gol', ' GOL', 'Gol'],
    'Yaris': [' Yaris', ' YARIS', ' Yari', ' yaris', ' yari', 'Yari', ' YARI', 'yaris'],
    'Q2': [' q2', ' Q2'],
    '2 Series': [' 2 series', ' 2 serie', '2 Series', ' 2 SERIES', ' 2 Serie', '2 Serie', ' 2 Series'],
    '3 Series': [' 3 Series', ' 3 Serie', ' 3 series', ' 3 SERIES', ' 3 serie', '3 Serie'],
    'A3': [' A3', ' a3'],
    'Octavia': [' Octavi', ' OCTAVIA', ' Octavia', ' octavia', 'Octavi', 'octavia'],
    'Passat': [' PASSAT', ' passat', ' Passa', 'Passat', ' Passat'],
    'Insignia': [' Insigni', ' INSIGNIA', ' insignia', ' Insignia', ' INSIGNI', 'Insigni'],
    'Fabia': [' Fabia', ' fabia', ' FABIA', ' Fabi', 'FABIA'],
    'A Class': [' A Clas', ' A Class', ' a class', ' A CLASS', 'a class', 'A CLASS', ' a clas'],
    'Ka+': [' Ka+', ' KA+', ' ka+', 'ka+'],
    'GLC Class': [' GLC Class', ' GLC CLASS', ' GLC Clas', ' glc class', ' glc clas'],
    'I30': [' i30', ' I30'],
    'C Class': [' C Clas', ' C CLASS', ' c class', 'C Clas', ' C CLAS', 'c class', ' c clas', ' C Class'],
    'Polo': [' POLO', ' Polo'],
    'E Class': [' E Class', ' E Clas', ' E CLASS', ' e class', 'E CLASS', 'e class'],
    'Q5': [' Q5', ' q5', 'q5'],
    'Up': ['U', ' up', ' UP', ' Up', ' U', 'UP'],
    'Fiesta': [' FIESTA', ' fiesta', ' Fiest', ' Fiesta', 'fiesta', 'Fiest', ' FIESTA'],
    'C-HR': [' C-H', ' c-hr', ' C-HR'],
    'Mokka X': [' mokka x', ' MOKKA X', ' Mokka X'],
    'Corsa': [' Corsa', ' corsa', ' Cors', ' CORSA', ' cors', ' CORS', 'corsa'],
    'Astra': [' ASTRA', ' Astr', ' Astra', ' astra', 'ASTRA', 'astra'],
    'TT': [' tt', ' TT', ' T'],
    '5 Series': [' 5 Series', ' 5 Serie', ' 5 SERIES', ' 5 series', '5 SERIES'],
    'Aygo': [' aygo', ' ayg', ' AYGO', ' Ayg', ' Aygo', 'aygo', ' AYG'],
    '4 Series': [' 4 SERIES', ' 4 Serie', ' 4 serie', '4 series', '4 Series', ' 4 Series', ' 4 series'],
    'SLK': [' slk', ' SLK'],
    'Viva': [' viva', ' Viva', ' VIVA', ' Viv'],
    'Focus': [' Focus', ' Focu', ' FOCUS', ' focus', ' FOCU', 'focus', 'Focu', ' focu', 'FOCUS'],
    'EcoSport': [' EcoSpor', ' ECOSPORT', ' ecosport', ' EcoSport'],
    'X-CLASS': [' x-clas', ' X-CLAS', ' x-class', ' X-CLASS'],
    'CL Class': [' cl class', ' CL Clas', ' CL CLASS', ' CL Class'],
    'IX20': [' ix20', ' IX20'],
    'Rapid': [' Rapi', ' rapid', ' Rapid'],
    'Auris': [' Auris', ' AURIS', ' auris', ' Auri'],
    'I20': [' i20', ' I20'],
    'X3': [' x3', ' X3'],
    'A8': [' A8', 'a8'],
    'GLS Class': [' GLS Clas', ' GLS CLASS', ' gls class', ' GLS Class'],
    'B-MAX': [' B-MA', ' B-MAX', 'B-MA', ' b-max'],
    'A4': [' A4', ' a4'],
    'Kona': [' KONA', ' Kon', ' Kona', ' KON', ' kona'],
    'I10': [' i10', ' I10'],
    'A1': [' A1', ' a1'],
    'Mokka': [' Mokka ', ' Mokk', ' Mokka', ' mokka ', ' mokka', ' MOKKA', 'Mokka ', 'Mokk'],
    'S-MAX': [' S-MA', ' s-max', ' S-MAX', ' s-ma'],
    'X2': [' x2', ' X2'],
    'Crossland X': [' crossland x', ' CROSSLAND X', ' Crossland X'],
    'Tiguan': [' Tiguan', ' tiguan', ' Tigua', ' TIGUAN', ' TIGUA', 'Tigua', 'TIGUAN', 'tiguan'],
    'A5': [' A5', ' a5'],
    'GLE Class': [' GLE Clas', ' GLE Class', ' gle class', ' GLE CLASS'],
    'Zafira': [' Zafira', ' Zafir', ' ZAFIRA', ' zafira', 'Zafir'],
    'Ioniq': [' Ioni', ' Ioniq', ' IONIQ', 'IONIQ', ' ioniq'],
    'A6': [' A6', ' a6'],
    'Yeti Outdoor': [' yeti outdoor', ' Yeti Outdoor'],
    'X1': [' x1', 'x1', ' X1'],
    'Scala': [' SCALA', ' Scala', ' scala', ' Scal', ' scal'],
    'S Class': [' S Class', ' S Clas', ' s class', ' S CLASS'],
    '1 Series': [' 1 Series', ' 1 SERIES', ' 1 Serie', ' 1 series', '1 SERIES', ' 1 SERIE', '1 series', ' 1 serie'],
    'Kamiq': [' KAMIQ', ' KAMI', ' kamiq', ' Kamiq'],
    'Kuga': [' Kug', ' KUGA', ' kuga', 'Kuga', ' Kuga', 'kuga'],
    'Tourneo Connect': [' tourneo connect', ' Tourneo Connect'],
    'Q7': [' q7', ' Q7'],
    'GLA Class': [' GLA Class', ' GLA CLASS', ' GLA Clas', ' gla class'],
    'Arteon': [' arteon', ' Arteon'],
    'SL CLASS': [' SL CLAS', ' SL CLASS', ' sl class', ' SL'],
    'Tucson': [' Tucson', ' TUCSON', ' Tucso', ' tucson', ' TUCSO', 'Tucso'],
    'Santa Fe': [' Santa F', ' santa fe', ' SANTA FE', ' Santa Fe'],
    'Grandland X': [' Grandland X', ' grandland x', ' GRANDLAND X'],
    'RAV4': [' rav4', 'RAV4', 'RAV', ' RAV', 'rav4', ' rav', ' RAV4'],
    'Touran': [' Touran', 'Toura', ' TOURAN', ' touran', ' Toura', ' TOURA'],
    'Citigo': [' Citig', ' citigo', ' Citigo', ' CITIGO'],
    'Roomster': [' Roomste', ' Roomster'],
    'Prius': [' PRIUS', ' Prius', 'Prius', ' prius'],
    'Corolla': [' corolla', ' COROLLA', ' Coroll', ' Corolla', 'corolla'],
    'B Class': [' b class', ' B Clas', ' B Class', ' B CLASS', 'b class'],
    'Sharan': [' sharan', ' Shara', ' Sharan', ' SHARAN'],
    'Kodiaq': [' Kodia', ' kodiaq', 'kodiaq', ' KODIAQ', ' Kodiaq'],
    'V Class': [' V Clas', ' V CLASS', ' V Class'],
    'Caddy Maxi Life': [' Caddy Maxi Lif', ' Caddy Maxi Life'],
    'Superb': [' Superb', ' Super', ' SUPERB', ' superb', ' super'],
    'T-Roc': [' T-Roc', ' T-RO', ' t-roc', ' T-Ro', ' T-ROC'],
    'Combo Life': [' COMBO LIFE', ' combo life', ' Combo Lif', 'COMBO LIFE', ' Combo Life'],
    'Beetle': [' Beetl', ' Beetle', ' beetle'],
    'Galaxy': [' GALAXY', ' Galax', ' galaxy', ' Galaxy'],
    'M3': [' M3', ' m3'],
    'GTC': [' gtc', ' GTC', ' gtc'],
    'X4': [' X4'],
    'KA': [' Ka', ' ka', ' K', ' KA'],
    'IX35': [' ix35', ' IX35'],
    'Grand Tourneo Connect': [' Grand Tourneo Connec', ' Grand Tourneo Connect'],
    'M4': [' m4', ' M4'],
    'Tourneo Custom': [' tourneo custom', ' Tourneo Custo', ' Tourneo Custom'],
    'Z4': [' Z4', ' z4'],
    'X5': [' X5', ' x5'],
    'Meriva': [' Meriva', ' MERIVA', ' Meriv', ' meriva'],
    'yeti outdoor': [' YETI OUTDOOR', ' Yeti Outdoo', ' yeti outdoor', 'yeti outdoor'],
    'RS6': [' RS6'],
    'Verso': [' VERSO', ' verso', ' Verso', ' Vers'],
    'Touareg': [' Touareg', ' TOUAREG', ' touareg', ' Touare'],
    'Mondeo': [' MONDEO', ' Mondeo', ' mondeo', ' MONDE', ' Monde'],
    'Shuttle': [' shuttle', ' Shuttle'],
    'CLS Class': [' CLS Class', ' cls class', ' CLS Clas', ' CLS CLASS'],
    'C-MAX': [' C-MAX', ' c-max', ' C-MA'],
    'Puma': [' puma', ' PUMA', ' Puma'],
    'CLA Class': [' CLA Class', ' CLA CLASS', ' cla class'],
    'I40': [' I40', ' i40'],
    'Q3': [' q3', ' Q3'],
    'Tiguan Allspace': [' TIGUAN ALLSPACE', ' tiguan allspace', ' Tiguan Allspac', ' Tiguan Allspace'],
    '6 Series': [' 6 SERIES', ' 6 series', ' 6 Series', ' 6 Serie'],
    'Caravelle': [' caravelle', ' Caravell', ' Caravelle'],
    'Karoq': [' Karoq', ' karoq', ' KAROQ', ' Karo'],
    'I3': [' i3', 'i3', ' I3'],
    'Grand C-MAX': [' GRAND C-MAX', ' grand c-max', ' Grand C-MA', ' Grand C-MAX'],
    'T-Cross': [' T-Cros', ' T-CROSS', ' T-Cross', ' t-cros', ' t-cross'],
    'A7': [' a7', ' A7'],
    'Golf SV': [' Golf SV', ' golf sv', ' GOLF SV'],
    'A': [' a', ' A'],
    'GT86': [' gt86', ' GT86'],
    'Yeti': [' yeti', ' Yet', ' Yeti'],
    'X': [' x', ' X'],
    'Land Cruiser': [' Land Cruise', ' Land Cruiser'],
    'EDGE': [' Edge', ' edge', ' Edg', ' EDGE'],
    'X6': [' X6'],
    'Fusion': [' Fusion'],
    'GL Class': [' GL CLASS', ' gl class', ' GL Class'],
    'Scirocco': [' scirocco', ' SCIROCCO', ' Scirocc', ' Scirocco'],
    'Z3': [' Z3'],
    'Hilux': [' hilux', ' Hilux', ' Hilu'],
    'Amarok': [' amarok', ' Amarok', ' Amaro'],
    'CC': [' cc', ' CC'],
    '7 Series': [' 7 Serie', ' 7 SERIES', ' 7 series', ' 7 Series'],
    'Avensis': [' AVENSIS', ' avensis', ' Avensis'],
    'M Class': [' m class', ' M CLASS', ' M Class', ' M Clas'],
    'Grandland': [' grandland ', ' Grandland '],
    'Zafira Tourer': [' Zafira Toure', ' ZAFIRA TOURER', ' Zafira Tourer'],
    'R8': [' R8', ' r8'],
    'Mustang': [' mustang', ' Mustang'],
    'Q8': [' Q8'],
    'CLK': [' CLK'],
    'RS3': [' RS3'],
    'Jetta': [' JETTA', ' Jetta'],
    'Supra': [' Supra'],
    'X7': [' X7'],
    'SQ7': [' SQ7'],
    'S3': [' s3'],
    'Arteon': [' Arteo', 'Arteo', ' ARTEON'],
    'GLB Class': [' glb class', ' GLB Class'],
    'Adam': [' Ada', ' adam', ' ADAM', ' Adam'],
    'M5': [' M5'],
    'Golf S': [' golf s', ' Golf S'],
    'VECTRA': ['Vectra', ' Vectra', ' VECTRA'],
    '8 Series': [' 8 SERIES', ' 8 Serie', ' 8 Series'],
    'Urban Cruiser': [' Urban Cruise', ' Urban Cruiser'],
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_model.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_val["model"] = X_val["model"].replace(replacement_dict)

# Optional: verify the cleaning
print(X_val["model"].unique())

#for model in sorted(X_val["model"].dropna().astype(str).unique()):
   # print(model)

In [ ]:
#corrections of 'transmission' for X_train

correct_transmission = {
    'Semi-Auto': ['Semi-Aut', 'semi-auto', 'emi-Auto', 'SEMI-AUTO', 'SEMI-AUT', 'EMI-AUTO', 'emi-Aut', 'emi-auto', 'semi-aut'],
    'Manual': ['anual', 'manual', 'Manua', 'MANUAL', ' Manual ', 'ANUAL', 'manua', 'anua', 'MANUA', ' manual ', ' MANUAL ', ' Manual', 'Manual ', 'manual '],
    'Automatic': ['AUTOMATIC', 'automatic', 'Automati', 'utomatic', 'UTOMATIC', 'automati', 'AUTOMATI', 'utomati'],
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_transmission.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_train["transmission"] = X_train["transmission"].replace(replacement_dict)

#replacing the unknown variable with a missing value
X_train["transmission"] = X_train["transmission"].replace(['unknow','UNKNOWN','nknown','nknow', 'unknown'], np.nan)
##X_train["transmission"] = X_train["transmission"].replace(['Other'], np.nan)

# Optional: verify the cleaning
print(X_train["transmission"].unique())

In [ ]:
#corrections of 'transmission' for X_val

correct_transmission = {
    'Semi-Auto': ['Semi-Aut', 'semi-auto', 'emi-Auto', 'SEMI-AUTO', 'SEMI-AUT', 'EMI-AUTO', 'emi-Aut', 'emi-auto', 'semi-aut'],
    'Manual': ['anual', 'manual', 'Manua', 'MANUAL', ' Manual ', 'ANUAL', 'manua', 'anua', 'MANUA', ' manual ', ' MANUAL ', ' Manual', 'Manual ', 'manual '],
    'Automatic': ['AUTOMATIC', 'automatic', 'Automati', 'utomatic', 'UTOMATIC', 'automati', 'AUTOMATI', 'utomati'],
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_transmission.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_val["transmission"] = X_val["transmission"].replace(replacement_dict)

#replacing the unknown variable with a missing value
X_val["transmission"] = X_val["transmission"].replace(['unknow','UNKNOWN','nknown','nknow', 'unknown'], np.nan)
#X_train["transmission"] = X_train["transmission"].replace(['Other'], np.nan)

# Optional: verify the cleaning
print(X_val["transmission"].unique())

For the X_train and X_val, in the 'tramission' variable , we replace the 'unknown' values by missing values to fill them later.

In [ ]:
#corrections of 'fuelType' for X_train

correct_fuelType = {
    'Petrol': ['etrol', 'petrol', 'PETROL', 'Petro', 'petro', 'ETROL', 'PETRO', 'etro', 'ETRO'],
    'Diesel': ['diesel','iesel','Diese','DIESEL','DIESE','IESEL','iese','diese','IESE'],
    'Hybrid': ['HYBRID','ybri','Hybri','ybrid','hybrid','YBRID','HYBRI'],
    'Other': ['ther','Othe','OTHER','other']
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_fuelType.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_train["fuelType"] = X_train["fuelType"].replace(replacement_dict)

#replacing the unknown variable with a missing value
# ?????? 
X_train["fuelType"] = X_train["fuelType"].replace(['ther','Othe','OTHER','other', 'Other'], np.nan)

# Optional: verify the cleaning
print(X_train["fuelType"].unique())


In [ ]:
#corrections of 'fuelType' for X_val

correct_fuelType = {
    'Petrol': ['etrol', 'petrol', 'PETROL', 'Petro', 'petro', 'ETROL', 'PETRO', 'etro', 'ETRO'],
    'Diesel': ['diesel','iesel','Diese','DIESEL','DIESE','IESEL','iese','diese','IESE'],
    'Hybrid': ['HYBRID','ybri','Hybri','ybrid','hybrid','YBRID','HYBRI'],
    'Other': ['ther','Othe','OTHER','other']
}

# Create a reverse lookup dictionary (each incorrect form maps to the correct one)
replacement_dict = {variant: correct for correct, variants in correct_fuelType.items() for variant in variants}

# Replace incorrect brand names with the correct ones
X_val["fuelType"] = X_val["fuelType"].replace(replacement_dict)

#replacing the unknown variable with a missing value
# ?????? 
X_train["fuelType"] = X_train["fuelType"].replace(['ther','Othe','OTHER','other', 'Other'], np.nan)

# Optional: verify the cleaning
print(X_val["fuelType"].unique())

For the X_train and X_val, in the fuelType variable , we replace the 'other' values by missing values to fill them later, since the only options that appear in the metadata is (Diesel, Petrol, Hybrid, Electric). 

In [ ]:
#checking the unique values after alterations
for col in X_train:
    print(f"\nColumn: {col}")
    print(X_train[col].unique())

In [ ]:
#checking the unique values after alterations
for col in X_val:
    print(f"\nColumn: {col}")
    print(X_val[col].unique())

#### Correction of incoherent values for numerical variables:

In [ ]:
df_X_train = pd.DataFrame(X_train)

In [ ]:
df_X_val = pd.DataFrame(X_val)

In [ ]:
df_X_train

In [ ]:
df_X_val

In [ ]:
df_X_train.describe().T

In [ ]:
df_X_val.describe().T

First, we decided to analyse the collumn previousOwners, that had negative values

In [ ]:
df_X_train[df_X_train['previousOwners'] < 0]['previousOwners']

In [ ]:
df_X_val[df_X_val['previousOwners'] < 0]['previousOwners']

The dataset has 265 cars with negative owners, all equal to -2, which is not possible. Since the mean value of the previousOwners is 1.994580 and the median is 2, we made the assumption that these negative values are spelling mistakes made during the data collection, that added the '-'. Therefore, we decided the change all of the negative values to positive

In [ ]:
df_X_train['previousOwners'] = df_X_train['previousOwners'].replace(-2, 2)

In [ ]:
df_X_val['previousOwners'] = df_X_val['previousOwners'].replace(-2, 2)

In [ ]:
df_X_train.describe().T

In [ ]:
df_X_val.describe().T

The variable mileage, the mpg and the engineSize present negative values, which is also impossible.

##### VARIABLE MILEAGE

In [ ]:
df_X_train[df_X_train['mileage'] < 0]['mileage']

In [ ]:
df_X_val[df_X_val['mileage'] < 0]['mileage']

Here we have 247 negative values. After reviewing the boxplots we decided that it could also be due to spelling mistakes in the data collection, since their absolute value did make sense in the variable range of values, so we converted the values.

In [ ]:
df_X_train['mileage'] = df_X_train['mileage'].abs()

In [ ]:
df_X_val['mileage'] = df_X_val['mileage'].abs()

In [ ]:
df_X_train[df_X_train['mileage'] < 0]['mileage']

##### VARIABLE MPG

In [ ]:
df_X_train[df_X_train['mpg'] < 0]

In [ ]:
df_X_val[df_X_val['mpg'] < 0]

In this table we have 25 rows and we can observe that the negative values are all the same. In the context of the variable 'mpg' is not normal to have 25 cars with the exact same number of milles per gallon, since the car's consumption depends a lot on several factors such as the driver, the age of the car and the way the car is used. We have also noticed that all of the cars with negative values are BMWs, so it was probably an error related to the brand. Therefore we decided to replace them with missing values to fill them later.

In [ ]:
df_X_train.loc[df_X_train["mpg"] < 0, "mpg"] = np.nan

In [ ]:
df_X_val.loc[df_X_val["mpg"] < 0, "mpg"] = np.nan

In [ ]:
df_X_train.describe().T

In [ ]:
df_X_val.describe().T

##### VARIABLE ENGINESIZE

In [ ]:
df_X_train[df_X_train['engineSize'] < 0]

In [ ]:
df_X_val[df_X_val['engineSize'] < 0]

For the variable engineSize we have 67 negative values, all with the same value. It makes sense for this variable, we can have several cars with the same engine type. After reviewing the boxplots we decided that it could also be due to spelling mistakes in the data collection, their absolute value did make sense in the variable range of values, so we converted these values to their absolute value.

In [ ]:
df_X_train['engineSize'] = df_X_train['engineSize'].abs()

In [ ]:
df_X_val['engineSize'] = df_X_val['engineSize'].abs()

In [ ]:
df_X_train[df_X_train['engineSize'] < 0]['engineSize']

In [ ]:
df_X_val[df_X_val['engineSize'] < 0]['engineSize']

##### VARIABLE TAX

In [ ]:
df_X_train[df_X_train['tax'] < 0]

In [ ]:
df_X_val[df_X_val['tax'] < 0]

In [ ]:
df_X_train['tax'] = df_X_train['tax'].abs()

In [ ]:
df_X_val['tax'] = df_X_val['tax'].abs()

### Removing Duplicates

In [ ]:
#checking duplicates again after handling Incoherencies
df_X_train.duplicated().sum()

In [ ]:
df_X_val.duplicated().sum()

In [ ]:
df_X_train = df_X_train.drop_duplicates()

In [ ]:
df_X_val = df_X_val.drop_duplicates()

### Treating outliers

### Treating missing values

## 5. Modelling

## 6. Acessment

## 7. Deployment